# Import

In [ ]:
import random
import numpy as np
import pandas as pd
from main import ArduinoReader
from motion_stages import Zaber_2axis_LST1500D as motion_stage
import matplotlib.pyplot as plt
import seaborn as sns


# Random Positions
The code snippet provided performs a series of measurements at random positions on a grid. It first initializes an instance of a motion stage and zeros its axes. Additionally, it establishes communication with an Arduino sensor reader through a specified COM port.

The "iterations" variable indicates the number of random measurements the code will perform. In the provided example, it is set to perform 100 iterations.

Within each iteration, the code generates a random x and y position within a 30 cm by 30 cm grid (hence, the values range from 0 to 300 mm). The motion stage is then commanded to move to the generated x and y position. Subsequently, it queries the Arduino sensor reader for the current sensor readings, which should represent the displacement of the sensor in the x and y directions from its starting point.

Each iteration's stage position and corresponding sensor readings are recorded and appended to a pandas DataFrame. The DataFrame is structured to consist of four columns:`Stage X (mm)`, `Stage Y (mm)`, `Sensor X`, and `Sensor Y`. Each row in this DataFrame corresponds to a single measurement iteration.

After all the iterations are completed, the DataFrame containing the recorded data is saved as an Excel file named `random_positions.xlsx`. The index parameter in the to_excel method is set to False, which means the index of the DataFrame will not be written to the Excel file.

Finally, the communication with the Arduino sensor reader is terminated using the `stop` method.

As a result of running this code, you can expect to have an Excel file where each row contains a randomly chosen stage position and the corresponding sensor readings. This data can be used to analyze the behavior of the sensor over a randomly sampled subset of the entire grid.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# initialize stage instance and zero axes
stage = motion_stage()
stage.home_axes()

# initialize ArduinoReader
arduino = ArduinoReader('COM3', 9600)  # replace with your port

# Set the number of iterations
iterations = 100

# Initialize center of the stage
x_center = 150  # in mm
y_center = 150  # in mm

# Re-initialize the sensor at the new origin
stage.move_x_absolute(x_center)
stage.move_y_absolute(y_center)
arduino.resetDistance()

# Initialize dataframe
df = pd.DataFrame(columns=["Stage X (mm)", "Stage Y (mm)", "Sensor X", "Sensor Y"])

for i in range(iterations):
    # move to a random x and y position within 30cm of the center, in mm
    x = random.uniform(x_center - 150, x_center + 150)
    y = random.uniform(y_center - 150, y_center + 150)

    stage.move_x_absolute(x)
    stage.move_y_absolute(y)

    # Record the x and y readings from the sensor
    sensor_x, sensor_y = arduino.distanceSinceInit()

    # add to dataframe
    df.loc[i] = [x, y, sensor_x, sensor_y]

    # print to console
    print(f"Iteration {i + 1}/{iterations}")
    print(f"Stage X (mm): {x}, Stage Y (mm): {y}, Sensor X: {sensor_x}, Sensor Y: {sensor_y}")
    print("------------------------")

# save to Excel
df.to_excel("random_positions.xlsx", index=False)

# Plotting
plt.figure(figsize=(20, 5))

# Stage X vs Sensor X
plt.subplot(1, 4, 1)
plt.scatter(df["Stage X (mm)"], df["Sensor X"])
plt.xlabel("Stage X (mm)")
plt.ylabel("Sensor X")
plt.title("Stage X vs Sensor X")

# Stage Y vs Sensor Y
plt.subplot(1, 4, 2)
plt.scatter(df["Stage Y (mm)"], df["Sensor Y"])
plt.xlabel("Stage Y (mm)")
plt.ylabel("Sensor Y")
plt.title("Stage Y vs Sensor Y")

# Stage X vs Stage Y
plt.subplot(1, 4, 3)
plt.scatter(df["Stage X (mm)"], df["Stage Y (mm)"])
plt.xlabel("Stage X (mm)")
plt.ylabel("Stage Y (mm)")
plt.title("Stage X vs Stage Y")

# Sensor X vs Sensor Y
plt.subplot(1, 4, 4)
plt.scatter(df["Sensor X"], df["Sensor Y"])
plt.xlabel("Sensor X")
plt.ylabel("Sensor Y")
plt.title("Sensor X vs Sensor Y")

plt.tight_layout()
plt.show()

# More detailed analysis using seaborn pairplot
sns.pairplot(df)

# Display statistics
print(df.describe())

# cleanup
arduino.stop()

# Incremental Positioning
This code allows you to perform an experiment where a motion stage is moved to various positions on a grid, and at each position, the readings from a sensor are recorded.

Firstly, the motion stage is initialized and homed to ensure a defined starting point. The sensor is also set up for reading its data through the ArduinoReader class.

The `step_size` variable specifies the distance between the points on the grid, and the `x_values` and `y_values` arrays are the coordinates of these points. For example, with a step size of 10 mm, the stage will move to every point (0,0), (0,10), (0,20), ... , (300,300) in the 300 x 300 mm area.

The `df` is a pandas DataFrame, which is a type of table where the results of the experiment will be stored. Each row corresponds to a different point on the grid, and each column corresponds to a different measurement: the x and y coordinates of the stage, and the x and y readings from the sensor.

The code then iterates over all the points on the grid. For each point, it moves the stage to the corresponding position, reads the sensor's current readings, and adds these to the DataFrame. 

After all points have been visited, the DataFrame is saved to an Excel file, named "incremental_positioning.xlsx", which can be opened for further analysis. This way, you'll have a detailed record of how the sensor's readings vary over the entire grid, which could provide valuable information about the characteristics of the system or the material being examined.

Finally, the `arduino.stop()` method is called to properly close the connection to the Arduino. 

Remember, each time you run this code, it will overwrite the previous Excel file, so be sure to rename or move the file if you wish to keep the previous data. Also, keep in mind that the sensor's readings are cumulative, so if you want to reset the readings to zero at the start of each experiment, you can call `arduino.reset()`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# initialize stage instance and zero axes
stage = motion_stage()
stage.home_axes()

# initialize ArduinoReader
arduino = ArduinoReader('COM3', 9600)  # replace with your port and baud rate

# Set up the center of the grid and the range
x_center = 150  # in mm
y_center = 150  # in mm
range_ = 150  # in mm

# Set up the grid
step_size = 10  # step size in mm
x_values = np.arange(x_center - range_/2, x_center + range_/2 + 1, step_size)  # x coordinates
y_values = np.arange(y_center - range_/2, y_center + range_/2 + 1, step_size)  # y coordinates

# Initialize an array to hold all the readings
readings = np.zeros((len(x_values), len(y_values), 2))

# Reset sensor readings at the new origin
arduino.reset()

# iterate over the grid
for i, x in enumerate(x_values):
    for j, y in enumerate(y_values):
        stage.move_x_absolute(x)
        stage.move_y_absolute(y)

        # Record the x and y readings from the sensor
        sensor_x, sensor_y = arduino.distanceSinceInit()

        # Save the readings
        readings[i, j, :] = sensor_x, sensor_y

        # Print the readings in the console
        print(f"Stage X: {x}, Stage Y: {y}, Sensor X: {sensor_x}, Sensor Y: {sensor_y}")

# save the readings to Excel
df = pd.DataFrame(readings.reshape(-1, 2), columns=["Sensor X", "Sensor Y"])
df.to_excel("incremental_positioning.xlsx", index=False)

# Calculate and print some basic statistics
print(df.describe())

# Plot histograms of the readings
df.hist()
plt.show()

# 2D plots of Stage X vs Stage Y and Sensor X vs Sensor Y
sns.jointplot(x_values, y_values, kind="hex")
plt.xlabel("Stage X (mm)")
plt.ylabel("Stage Y (mm)")
plt.title("Stage X vs Stage Y")

sns.jointplot(df["Sensor X"], df["Sensor Y"], kind="hex")
plt.xlabel("Sensor X")
plt.ylabel("Sensor Y")
plt.title("Sensor X vs Sensor Y")

plt.tight_layout()
plt.show()

# cleanup
arduino.stop()

# Multiple Runs
The provided Python code block works in conjunction with an Arduino-based optical sensor and a 2-axis motion stage to measure and record the positional variance of the sensor in the X and Y dimensions over a 2D grid.

Upon execution, the program first initializes the motion stage and sets it to its zero (home) position. Simultaneously, an ArduinoReader instance is launched to monitor the sensor's output.

Next, a 2D grid is generated to guide the motion stage, and an empty 3D array is created to store sensor readings. For each run, the stage systematically traverses the grid by moving to each (X, Y) coordinate and pausing to take a sensor reading. The ArduinoReader records the X and Y position data from the sensor at each point and stores them in the designated array.

This process is repeated for a predetermined number of runs, providing multiple sensor readings for each point on the grid. Upon completion of all runs, the program calculates the mean and standard deviation for each grid point's sensor readings, offering insights into the consistency and reliability of the sensor's performance across the 2D grid.

The results (mean and standard deviation of sensor readings at each grid point) are then stored in a Pandas DataFrame and exported to an Excel spreadsheet for easy viewing and further analysis. This document provides an organized record of the sensor's performance across multiple runs, essential for identifying patterns, irregularities, or potential issues.

Finally, the program calls the ArduinoReader's stop method to end the sensor reading process, cleaning up resources to ensure the program ends cleanly.

To use this code, replace `COM3` and 9600 in the ArduinoReader initialization with your Arduino's port and baud rate. Modify the number of runs, the step size, and the range of X and Y values as per your requirements. After running this program, you can open the "multiple_runs.xlsx" file to view the sensor's mean and standard deviation at each grid point over the multiple runs.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random

from motion_stages import Zaber_2axis_LST1500D as motion_stage
from arduino_reader import ArduinoReader

# initialize stage instance and zero axes
stage = motion_stage()
stage.home_axes()

# initialize ArduinoReader
arduino = ArduinoReader('COM3', 9600)  # replace with your port and baud rate

# Set the origin to be the center of the stage
x_center = 150  # in mm
y_center = 150  # in mm

stage.move_x_absolute(x_center)
stage.move_y_absolute(y_center)

# Reinitialize the sensor at the new origin
arduino.reset()

# Set up the grid
step_size = 10  # step size in mm
x_values = np.arange(x_center-150, x_center+151, step_size)  # x coordinates
y_values = np.arange(y_center-150, y_center+151, step_size)  # y coordinates

# Set up number of runs
num_runs = 5

# Initialize a 3D array to hold all the runs
runs = np.zeros((num_runs, len(x_values), len(y_values), 2))

# Perform the runs
for run in range(num_runs):
    print(f"Starting run {run + 1}")
    
    # iterate over the grid
    for i, x in enumerate(x_values):
        for j, y in enumerate(y_values):
            stage.move_x_absolute(x)
            stage.move_y_absolute(y)

            # Record the x and y readings from the sensor
            sensor_x, sensor_y = arduino.distanceSinceInit()

            # Save the readings
            runs[run, i, j, :] = sensor_x, sensor_y

# Calculate the mean and standard deviation of the readings at each position
means = np.mean(runs, axis=0)
std_devs = np.std(runs, axis=0)

# save the means and standard deviations to Excel
df_means = pd.DataFrame(means.reshape(-1, 2), columns=["Mean Sensor X", "Mean Sensor Y"])
df_std_devs = pd.DataFrame(std_devs.reshape(-1, 2), columns=["Std Dev Sensor X", "Std Dev Sensor Y"])
df = pd.concat([df_means, df_std_devs], axis=1)
df.to_excel("multiple_runs.xlsx", index=False)

# Print the DataFrame to console
print(df)

# Print statistical details to console
print(df.describe())

# Plotting
# Heatmap of sensor X readings
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
sns.heatmap(df[["Mean Sensor X", "Std Dev Sensor X"]])
plt.title("Sensor X Readings")
plt.show()

# Heatmap of sensor Y readings
plt.subplot(1, 2, 2)
sns.heatmap(df[["Mean Sensor Y", "Std Dev Sensor Y"]])
plt.title("Sensor Y Readings")
plt.show()

# Scatter plots
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.scatter(x_values, y_values, c='b', label='Stage')
plt.scatter(df["Mean Sensor X"], df["Mean Sensor Y"], c='r', label='Sensor')
plt.xlabel("X (mm)")
plt.ylabel("Y (mm)")
plt.title("Stage vs Sensor")
plt.legend()

# Stage vs Sensor spread
plt.subplot(1, 2, 2)
plt.scatter(df["Mean Sensor X"], df["Std Dev Sensor X"], c='b', label='Sensor X')
plt.scatter(df["Mean Sensor Y"], df["Std Dev Sensor Y"], c='r', label='Sensor Y')
plt.xlabel("Mean")
plt.ylabel("Std Dev")
plt.title("Sensor Spread")
plt.legend()
plt.tight_layout()
plt.show()

# cleanup
arduino.stop()


# Reverse Movements

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# initialize stage instance and zero axes
stage = motion_stage()
stage.home_axes()

# initialize ArduinoReader
arduino = ArduinoReader('COM3', 9600)  # replace with your port and baud rate

# Set up the grid
step_size = 10  # step size in mm
x_values = np.arange(300, -1, -step_size)  # x coordinates
y_values = np.arange(300, -1, -step_size)  # y coordinates

# Initialize an array to hold all the readings
readings = np.zeros((len(x_values), len(y_values), 2))

# iterate over the grid
for i, x in enumerate(x_values):
    for j, y in enumerate(y_values):
        stage.move_x_absolute(x)
        stage.move_y_absolute(y)

        # Record the x and y readings from the sensor
        sensor_x, sensor_y = arduino.distanceSinceInit()

        # Save the readings
        readings[i, j, :] = sensor_x, sensor_y

        # Print the readings to console
        print(f"At position ({x},{y}), Sensor X: {sensor_x}, Sensor Y: {sensor_y}")

# Convert the readings to a pandas DataFrame
df = pd.DataFrame(readings.reshape(-1, 2), columns=["Sensor X", "Sensor Y"])

# Save the DataFrame to an Excel file
df.to_excel("reverse_movements.xlsx", index=False)

# Print summary statistics to the console
print(df.describe())

# Plot histograms of the sensor readings
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(df["Sensor X"], bins=20, alpha=0.5, color='r', label="Sensor X")
plt.xlabel("Sensor X Readings")
plt.ylabel("Frequency")
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(df["Sensor Y"], bins=20, alpha=0.5, color='b', label="Sensor Y")
plt.xlabel("Sensor Y Readings")
plt.ylabel("Frequency")
plt.legend()

plt.tight_layout()
plt.show()

# cleanup
arduino.stop()
